# Methoden-Workshop Machine Learning & AIA
1. **Inhaltsanalyse-Daten einlesen uns ansehen** mit der Python-Bibliothek für Data Management ``pandas``.
2. **Text in Features umwandeln** - mit ``sklearn``'s ``Vectorizer``.
3. **Classification Functions** auswählen und anwenden.
4. Datensatz in **Train und Test Set** aufteilen.
5. Die Funktion auf den Trainingsdaten **fitten** aka. **trainieren**.
6. **Evaluieren** - Die Predictions mit der (manuellen) Codierung auf dem **Test Set abgleichen**.

Von: [anke.stoll@hhu.de](mailto:anke.stoll@hhu.de) <br>
Last edit: 08.02.2021


Basic Tutorials:

- [Python Basics Tutorial](https://www.youtube.com/watch?v=eXBD2bB9-RA&list=PLQVvvaa0QuDeAams7fkdcwOGBpGdHpXln)

## 1. Von Excel nach Python und wieder zurück mit ``pandas``

In [ ]:
import pandas as pd #Abkürzung für Faule und aus Konvention.

#### Daten einlesen - .csv or .txt oder Excel

Wo liegt/wie heißt der Datensatz?

In [ ]:
df = pd.read_csv("Data Sets/HateSpeech_Tweets_DataSet.csv",
                 sep=",") #Ändern bei Excel auf ";")

#### Check out your data frame

In [ ]:
df.head(10) #Zeigt die ersten 10 Zeilen.

In [ ]:
df.tail(10) #Oder die letzen 10.

In [ ]:
len(df) #Anzahl der Zeilen.

In [ ]:
df["HateSpeech"].value_counts() #Überblick zu den Werten in einer Spalte.

Es gibt noch viel mehr Funktionen und Möglichkeiten, Daten mit der `pandas`-Bibliothek zu bearbeiten, zu analysieren und darzustellen! 

Diese beiden **Playlists auf Youtube** sind sehr gut, vor allem für Menschen ohne Informatik-Vorkenntnisse:

- [Data School Pandas Best Practice](https://www.youtube.com/watch?v=hl-TGI4550M&list=PL5-da3qGB5IBITZj_dYSFqnd_15JgqwA6)
- [Pandas and Data Science](https://www.youtube.com/watch?v=yzIMircGU5I&list=PL5-da3qGB5ICCsgW1MxlZ0Hq8LL5U3u9y) Schon etwas älter, inzwischen gibt es 1,2 kleine Änderungen in der Bibliothek.

## 2. Text zu Features - Unabhängigen Variablen programmieren

Die zweite super awesome Python Bibliothek heißt ``sklearn``. In dieser Bibliothek finden wir alles, was wir für ML brauchen. Auch das ML mit Text wird uns besonders einfach gemacht. In ``sklearn`` gibt es eine Unterabteilung ``feature_extraction.text``. Hier finden wir Funktionen, die uns das Umwandeln von Text zu Wort-Vektoren sehr einfach machen! Wir benutzen ``CountVectorizer`` and ``TfidfVectorizer``. Full documentation auf der [sklearn Website](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html).


- [Machine Learning mit sklearn](https://www.youtube.com/watch?v=elojMnjn4kk&list=PL5-da3qGB5ICeMbQuqbbCOQWcS6OYBr5A) -Schon älter, aber immer noch eins der besten Tutorials die es gibt.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

All vectorizers have several parameters that you can modify.

In [ ]:
vec = CountVectorizer(ngram_range=(1,1)) 
#Ich speicher den hier ab, damit ich nicht immer ausschreiben muss.

In [ ]:
Tweets_vec = vec.fit_transform(df["Tweet"]) 
#Wir wenden den Vectorizer auf der Spalte "Tweet" in unserem Datensatz an.

In [ ]:
Tweets_vec # 469 tweets und 2645 features (unigrams).

#### Was sind unsere Features (UVs)?

In [ ]:
vec.get_feature_names() #Das sind unsere Features

I used [this code from stuckoverflow](https://stackoverflow.com/questions/45805493/sorting-tfidfvectorizer-output-by-tf-idf-lowest-to-highest-and-vice-versa). Alternatively, you can try [this code](https://towardsdatascience.com/very-simple-python-script-for-extracting-most-common-words-from-a-story-1e3570d0b9d0) to get an overview of the most frequent words in your documents (before vectorization).

In [ ]:
features = vec.get_feature_names()

In [ ]:
sums = Tweets_vec.sum(axis=0) 

data = []

for col, term in enumerate(features):
    data.append( (term, sums[0,col] ))

df_ranks = pd.DataFrame(data, columns=['Feature','Count'])
df_ranks.sort_values('Count', ascending=False, inplace=True)

In [ ]:
df_ranks.head()

In [ ]:
df_ranks.tail(20) 

#### Jetzt Bigrams!

In [ ]:
vec = CountVectorizer(ngram_range=(2,2)) 

In [ ]:
Tweets_vec = vec.fit_transform(df["Tweet"]) 

In [ ]:
vec.get_feature_names() #Das sind unsere Features

## 3. Den Classifier programmieren

Wir jetzt nun Features (UVs, Wörter mit ihren Häufigkeiten) erstellt, als AV haben wir Hate Speech, YES or NO. Jetzt berechnen wir einen Zusammenhang, heißt, wir versuchen die Kategorie _Hate Speech_ durch unsere Feautures zu zu schätzen (klassifizieren).

Als erstes müssen wir uns für eine Schätzfunktion - eine _Classification Function_ - entscheiden. Und zwar aus der Auswahl, die in der `sklearn` zu finden ist. Hier eine kleine Auswahl von Funktionen, die sich in der Forschung als geeignet herausgestellt haben. Spolier: Meistens muss man durch Trail und Error gerausfinden, welche sich eignet.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC

In [ ]:
model = MultinomialNB()

#### Den Datensatz in Train und Test Set aufteilen

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = df["Tweet"]

In [ ]:
y =df["HateSpeech"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, #X is your pandas column with the documents.
                                                    y, #y is your pandas coulmn with category.
                                                    test_size=0.33,#Relative size of the test set. e.g. 33% 
                                                    random_state=42) #Chose a number to reproduce the split.

In [ ]:
len(X_train) #Anzahl der Tweets in train data

In [ ]:
X_train_vec = vec.fit_transform(X_train) #Do this ONLY ON X TRAIN !

In [ ]:
X_train_vec

In [ ]:
model.fit(X_train_vec, y_train) #Fit the model, what means training. 

In [ ]:
X_test_vec = vec.transform(X_test)# Mache Predictions auf dem Test Set für die Evaluation. 
# Das Test Set muss auch i Vectorformat transformiert werden. Benutze die funtion transform, nicht fit_transform!

In [ ]:
y_pred = model.predict(X_test_vec)

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

In [ ]:
#This is a nice overview of the performance of your model in all categories (on the test set).
print(classification_report(y_test, y_pred))

In [ ]:
print(confusion_matrix(y_test, y_pred))

In [ ]:
#Nice confusion matrix output as a pandas data frame.
pd.DataFrame(
    confusion_matrix(y_test, y_pred),
    columns=['Predicted NO', 'Predicted YES'],
    index=['True NO', 'True YES']
)

## Fertig!